## Python porting of  Bollinger Bands TradingView Indicator

>Developed by [@edyatl](https://github.com/edyatl) April 2023 <edyatl@yandex.ru>


In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//@version=5 
indicator(shorttitle="BB", title="Bollinger Bands", overlay=true, timeframe="", timeframe_gaps=true) 

length = input.int(20, minval=1) 
src    = input(close, title="Source") 
mult   = input.float(2.0, minval=0.001, maxval=50, title="StdDev") 

basis = ta.sma(src, length) 
dev = mult * ta.stdev(src, length) 

upper = basis + dev 
lower = basis - dev 

offset = input.int(0, "Offset", minval = -500, maxval = 500) 

plot(basis, "Basis", color=#FF6D00, offset = offset) 
p1 = plot(upper, "Upper", color=#2962FF, offset = offset) 
p2 = plot(lower, "Lower", color=#2962FF, offset = offset) 

fill(p1, p2, title = "Background", color=color.rgb(33, 150, 243, 95))
```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

# Nicest style for plots
# sns.set(style="ticks")

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="ATOMUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-04-30 21:45:00,11.64200000,11.65100000,11.59200000,11.59300000,10052.37000000,2023-04-30 21:59:59.999,116890.67371000,604,4289.29000000,49863.04461000,0
496,2023-04-30 22:00:00,11.59300000,11.59300000,11.55900000,11.57300000,10993.66000000,2023-04-30 22:14:59.999,127244.75981000,592,5921.93000000,68537.18450000,0
497,2023-04-30 22:15:00,11.57300000,11.57800000,11.55000000,11.57400000,8961.48000000,2023-04-30 22:29:59.999,103614.14500000,491,3750.26000000,43360.09534000,0
498,2023-04-30 22:30:00,11.57200000,11.57900000,11.54700000,11.56100000,5436.38000000,2023-04-30 22:44:59.999,62850.44086000,306,2305.98000000,26653.74588000,0
499,2023-04-30 22:45:00,11.56100000,11.58400000,11.56100000,11.58400000,1793.54000000,2023-04-30 22:59:59.999,20760.53655000,148,1018.87000000,11792.49245000,0


<IPython.core.display.Javascript object>

### Inputs

In [8]:
length: int = 20  # input.int(20, minval=1)
src: pd.Series = data["close"]  # input(close, title="Source")
mult: float = 2.0  # input.float(2.0, minval=0.001, maxval=50, title="StdDev")
# offset: int = 0  # input.int(0, "Offset", minval = -500, maxval = 500)

<IPython.core.display.Javascript object>

In [9]:
basis = tl.SMA(src, length)
dev = mult * tl.STDDEV(src, length)

upper = basis + dev
lower = basis - dev

<IPython.core.display.Javascript object>

In [10]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "Basis": basis,
        "Deviation": dev,
        "Upper": upper,
        "Lower": lower,
    }
)
res.tail(20)

,open_time,Basis,Deviation,Upper,Lower
480,2023-04-30 18:00:00,11.82435,0.064032,11.888382,11.760318
481,2023-04-30 18:15:00,11.82255,0.067689,11.890239,11.754861
482,2023-04-30 18:30:00,11.82240,0.067945,11.890345,11.754455
483,2023-04-30 18:45:00,11.82190,0.068361,11.890261,11.753539
484,2023-04-30 19:00:00,11.81665,0.079647,11.896297,11.737003
485,2023-04-30 19:15:00,11.80930,0.102982,11.912282,11.706318
486,2023-04-30 19:30:00,11.79930,0.122267,11.921567,11.677033
487,2023-04-30 19:45:00,11.78785,0.137125,11.924975,11.650725
488,2023-04-30 20:00:00,11.76785,0.183465,11.951315,11.584385
489,2023-04-30 20:15:00,11.75030,0.207380,11.957680,11.542920


<IPython.core.display.Javascript object>